In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import napari
from matplotlib import pyplot as plt  # graphic library, for plots
import numba as nb

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
from bokeh.sampledata.stocks import AAPL

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
import filedialogs

In [5]:
from metavision_core.event_io.raw_reader import RawReader
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io import EventsIterator

In [6]:
dt=1000

In [7]:
path = filedialogs.gui_fname('example_data/')
path = path.decode('ascii')

In [8]:
record_raw = RawReader(path)
print(record_raw)

RawReader(E:/code/github-ffvoigt/accordion/playground/example_data/fish_bouts_recording_2023-06-27_10-42-58.raw)
current time : 0us done : False
current event index : 0
_begin_buffer 0,_end_buffer_ 0,  buffer_size 10000000


In [9]:
events = record_raw.load_n_events(1000000)
print(events)

[(834, 596, 1,     8214) (925,   4, 0,     9194) (767, 127, 0,     9410)
 ... (829, 542, 0, 15191063) (771, 162, 0, 15191064)
 (810, 484, 0, 15191067)]


In [10]:
events['t'].max()

15191067

In [11]:
events['t'].max()/1000/1000

15.191066999999999

In [12]:
def calculate_bins(events, dt=1000):
    # calculate events for timestep dt in us
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    interval_us = max_timepoint-min_timepoint
    timepoints = int(interval_us / (dt))
    bins = np.linspace(min_timepoint, max_timepoint, timepoints)
    return bins

def calculate_bin_number(events, dt=1000):
    # calculate events for timestep dt in us
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    timepoints = int(np.floor((max_timepoint-min_timepoint) / (dt)))
    return timepoints

@nb.njit
def inside_circle(x, y, x_center, y_center, r):
    return np.power(x-x_center,2)+np.power(y-y_center,2) <= np.power(r,2)

def is_on_event(event):
    return event[2]==1

@nb.njit
def filter_array(arr, condition):
    result = np.empty_like(arr)
    j = 0
    for i in range(arr.size):
        if condition(arr[i]):
            result[j] = arr[i]
            j += 1
    return result[:j].copy()

@nb.njit
def events_inside_circular_roi(events, x_center, y_center, radius):
    result = np.empty_like(events)
    j = 0
    for i in range(events.size):
        if inside_circle(events[i]['x'],events[i]['y'],x_center,y_center, radius):
            result[j] = events[i]
            j += 1
    return result[:j].copy()

@nb.njit
def split_events(events):
    on_events = np.empty_like(events)
    off_events = np.empty_like(events)
    on_counter = 0
    off_counter = 0
    
    for i in range(events.size):
        if events[i][2]==1:
            on_events[on_counter] = events[i]
            on_counter += 1
        else: 
            off_events[off_counter] = events[i]
            off_counter += 1
   
    return on_events[:on_counter].copy(),off_events[:off_counter].copy()

In [13]:
inside_events = events_inside_circular_roi(events, 560, 375, 400)
#on_events, off_events = split_events(inside_events)

Note: The on/off edges are slightly different as the event timestamps are slightly different 
* might be solvable by shifting the first and the last events in time?
* or hardcoding the edges based on the original event data?

In [ ]:
def create_event_dict(events, dt=1000):
    on_events, off_events = split_events(events)
    event_dict = {}
    event_dict['on_events'] = on_events
    event_dict['off_events'] = off_events
    event_dict['metadata'] = {'num_events'    : len(events), 
                              'num_on_events' : len(on_events),
                              'num_off_events': len(off_events),
                             }
    bins = calculate_bins(events, dt)
    event_dict['on_hist']  , on_edges  = np.histogram(on_events['t'],  density=False, bins=bins)
    event_dict['off_hist'] , off_edges = np.histogram(off_events['t'], density=False, bins=bins)
    event_dict['on_edges'] = on_edges[1:]
    event_dict['off_edges'] = off_edges[1:]
    return event_dict

In [ ]:
myarray[2]

In [ ]:
on_off_filter = np.zeros(10, dtype=bool)

In [ ]:
on_off_filter[4] = True

In [ ]:
minievents = events[0:20]

In [ ]:
on_off_filter = np.zeros(len(minievents), dtype=bool)

In [ ]:
on_off_filter[4] = True
on_off_filter[5] = True
on_off_filter[6] = True

In [ ]:
minievents[on_off_filter]

In [ ]:
on_off_filter

In [ ]:
np.invert(on_off_filter)

In [ ]:
### API ideas

As usual: What datatype makes sense here? 

filters have len(events) and are bool vectors that can be used by np.filter

events.get_filtered_events(filter) # returns all events filtered by a filter
events.get_on_events(filter) # returns On events filtered by a filter 
events.get_off_events(filter) # returns Off events 

In [14]:
class Events():
   
    def __init__(self, events, dt=1000):
        # super().__init__()
        
        self.events = events
        self.dt = dt
        self.num_events =  len(self.events)
        
        # create on/off event filter:
        self.on_filter = self.initialize_filter()
        
        for i in range(len(events)):
            if events[i][2]==1:
                self.on_filter[i] = True
       
        self.num_on_events = np.sum(self.on_filter)
        self.num_off_events = self.num_events - self.num_on_events
        
        self.bins = self.calculate_bins(self.events, dt)
        self.num_bins = len(self.bins)
        
        self.on_hist , self.on_edges = np.histogram(self.get_on_events()['t'],  density=False, bins=self.bins)
        self.off_hist , self.off_edges = np.histogram(self.get_off_events()['t'],  density=False, bins=self.bins)
        self.on_edges = self.on_edges[1:]
        self.off_edges = self.off_edges[1:]
        
    def initialize_filter():
        return np.zeros(self.num_events, dtype=bool)
                
    def get_filtered_events(self, filter):
        if len(filter) == self.num_events:
            return self.events[filter]
        else:
            print('Number of elements in the filter does not equal number of events')

    def get_on_events(self):
        return self.events[self.on_filter]
    
    def get_off_events(self):
        return self.events[np.invert(self.on_filter)]
    
    def get_events_in_time_interval(self, t_start, t_stop):
        start_index = int(np.sum(self.events.on_hist[0:int(np.floor((t_start-self.events.on_edges[0])/dt))]))
        stop_index = int(np.sum(self.events.on_hist[0:int(np.ceil((t_stop-self.events.on_edges[0])/dt))]))
        return self.events[start_index:stop_index]
    
    def calculate_bins(self, events, dt=1000):
        # calculate events for timestep dt in us
        min_timepoint = self.events['t'].min()
        max_timepoint = self.events['t'].max()
        interval_us = max_timepoint-min_timepoint
        timepoints = int(interval_us / (dt))
        bins = np.linspace(min_timepoint, max_timepoint, timepoints)
        return bins
    
    def get_events_in_rect_roi(self, events, x_pos, y_pos, x_width, y_width):
        # how many events are in the bounding box?
        # create a filter vector for events and then apply it to the numpy array
        
        rect_roi_filter = self.initialize_filter()
        
        counter = 0
        for event in events:
            event_x_pos = event[1]
            event_y_pos = event[0]
            if x_pos < event_x_pos & event_x_pos < x_pos+x_width:
                if y_pos < event_y_pos & event_y_pos < y_pos+y_width:
                    counter+=1
        
        filtered_events = np.zeros(i)
    
        for event in events:
            event_x_pos = event[1]
            event_y_pos = event[0]
            if x_pos < event_x_pos & event_x_pos < x_pos+x_width:
                if y_pos < event_y_pos & event_y_pos < y_pos+y_width:
                    filtered_events = np.append(filtered_events, event)
                   
        return filtered_events
    
    def get_events_in_circ_roi(self, events, x_pos, y_pos, x_width, y_width)
        result = np.empty_like(events)
        j = 0
        for i in range(events.size):
            if inside_circle(events[i]['x'],events[i]['y'],x_center,y_center, radius):
                result[j] = events[i]
                j += 1
        return result[:j].copy()


In [15]:
myevents = Events(events, 1000)

In [ ]:
myevents.get_off_events()[9:10]

In [ ]:
event_dict = create_event_dict(events)

In [ ]:
event_dict.keys()

### Numpy arrays have names for columns!

In [ ]:
events.dtype.names

### Pyqtgraph Helper Tool

In [16]:
%gui qt5
from PyQt5.Qt import QApplication
import qdarkstyle
import pyqtgraph as pg

In [17]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [18]:
class EventExplorer():
    def __init__(self, events):
        self.ET_window = EventTimeSelector(events)
        self.ET_window.graphicsView.show()
                
        #self.EventXY_window = EventXYPlot(events)
        #self.EventXY_window.graphicsView.show()
        
        #self.EventYT_window = EventYTPlot(events)
        #self.EventYT_window.graphicsView.show()
        
        #self.TestWindow = TestWindow()
        #self.TestWindow.view.show()
                
class TestWindow():
    def __init__(self):
        self.view = pg.GraphicsView()   
        self.view.setWindowTitle('Event-Time-Selector')
        self.layout = pg.GraphicsLayout()
        self.view.setCentralItem(self.layout)
        
        # add subplots
        self.p0 = self.layout.addPlot(0,0)
        self.p0.addLegend()
        self.p0.plot([1,2,3,4,5], pen='b', name='p0')

        self.p1 = self.layout.addPlot(1,0)
        self.p1.addLegend()
        self.p1.plot([2,2,2,2,], pen='r', name='p1')

class EventTimeSelector():
    def __init__(self, events):
        self.events = events
        self.graphicsView = pg.GraphicsView()
        self.graphicsView.setWindowTitle('Event Time Selector')
        self.layout = pg.GraphicsLayout()
        self.graphicsView.setCentralItem(self.layout)
        #self.graphicsView.show()
        
        self.ET_plot = self.layout.addPlot(row=0, col=0)
        self.ET_region_selection_plot = self.layout.addPlot(row=1, col=0)
        
        self.ET_plot.plot(x = self.events.on_edges, y = self.events.on_hist, pen="b")
        self.ET_plot.plot(x = self.events.on_edges, y = self.events.on_hist+self.events.off_hist, pen="r")
        self.ET_region_selection_plot.plot(x = self.events.on_edges, y = self.events.on_hist+self.events.off_hist, pen="r")
        self.ET_region_selection_plot.setLabel('left', "Y Axis", units='Events')
        self.ET_region_selection_plot.setLabel('bottom', "X Axis", units='s')
        
        len_data = self.events.num_bins
        left_start_range = self.events.on_edges[int(len_data/3)]
        right_start_range = self.events.on_edges[int(2*len_data/3)]
        
        self.ET_region = pg.LinearRegionItem(values=[left_start_range,right_start_range])
        self.ET_region.setZValue(10) # Move item up
        self.ET_region_selection_plot.addItem(self.ET_region, ignoreBounds = True)
        self.ET_plot.setAutoVisible(y=True)
        
        #ET_plot.sigRangeChanged.connect(updateRegion) 
        self.ET_region.sigRegionChanged.connect(self.update)
        self.update()
        
    def updateRegion(self, window, viewRange):
        rgn = viewRange[0]
        self.ET_region_selection_plot.setRegion(rgn)
        
    def update(self):
        self.ET_region.setZValue(10)
        minT, maxT = self.ET_region.getRegion()
        self.ET_plot.setXRange(minT, maxT, padding=0)
        '''
        self.events_subset = self.event_time_slice(self.events['on_events'], minT, maxT)
        self.update_XY_plot(self.events_subset)
        self.update_YT_plot(self.events_subset)
        '''
        
class EventXYPlot():
    def __init__(self, events):
        self.events = events
        self.graphicsView = pg.GraphicsView()
        self.graphicsView.setWindowTitle('Event XY View')
        self.layout = pg.GraphicsLayout()
        self.graphicsView.setCentralItem(self.layout)
                
        self.XY_plot = self.layout.addPlot(row=0, col=0, rowspan=2, colspan=1)
        self.XY_plot.setAspectLocked(True, ratio=1.77)
        self.XY_plot.invertY(True)
            
        self.s4 = pg.ScatterPlotItem(
            size=5,
            pen=pg.mkPen(None),
            brush=pg.mkBrush(255, 255, 255, 20))

        self.s4.addPoints(x=self.events.on_events['x'],
            y=self.events.off_events['y'],
                # size=(np.random.random(n) * 20.).astype(int),
            #brush=[pg.mkBrush(x) for x in np.random.randint(0, 256, (len(on_events), 3))],
                #data=np.arange(n)
                )

        self.XY_plot.addItem(self.s4)     
        
        self.create_roi()
        
    def create_roi(self):
        self.roi = pg.RectROI([20, 20], [100, 100], pen=(0,9))
        #self.roi.addRotateHandle([1,0], [0.5, 0.5])
        self.XY_plot.addItem(self.roi)
        self.roi.sigRegionChangeFinished.connect(self.update_from_ROI)
        
    def updateRegion(self, window, viewRange):
        rgn = viewRange[0]
        # self.ET_region_selection_plot.setRegion(rgn)
        
    def update_from_ROI(self):
        self.roi_state = self.roi.getState()
        x_pos = self.roi_state['pos'].x()
        y_pos = self.roi_state['pos'].y()
        x_width = self.roi_state['size'].x()
        y_width = self.roi_state['size'].y()
        
        self.events_subset = self.rect_filter_events(self.events, x_pos, y_pos, x_width, y_width)
        self.update_YT_plot(self.events_subset)
        
    
    def update_XY_plot(self, events_subset):
        self.s4.clear()
        self.s4.addPoints(x=events_subset['x'],
                y=-events_subset['y']+720,
                     )
        
class EventYTPlot():
    def __init__(self, eventdict):
        self.events = eventdict
        self.graphicsView = pg.GraphicsView()
        self.graphicsView.setWindowTitle('Event YT View')
        self.layout = pg.GraphicsLayout()
        self.graphicsView.setCentralItem(self.layout)
        
        self.YT_plot = self.layout.addPlot(row=0, col=3)
        self.YT_plot.invertY(True)
        self.YT_plot.enableAutoRange()
        self.s5 = pg.ScatterPlotItem(
            size=3,
            pen=pg.mkPen(None),
            brush=pg.mkBrush(255, 255, 255, 20))
        
        self.YT_plot.addItem(self.s5)
        
    def update_YT_plot(self, events_subset):
        self.s5.clear()
        self.s5.addPoints(x=events_subset['t'],
                y=events_subset['y'],
                     )

In [19]:
myexplorer2 = EventExplorer(myevents)

In [ ]:
x_pos = myexplorer2.roi_state['pos'].x()
y_pos = myexplorer2.roi_state['pos'].y()
x_width = myexplorer2.roi_state['size'].x()
y_width = myexplorer2.roi_state['size'].y()

In [ ]:
filtered_events = np.array([])

In [ ]:
np.append(filtered_events, [4,6])

In [ ]:
myexplorer2.events['on_events'].shape

In [ ]:
myexplorer2.events['on_events'][0]

In [ ]:
True & True

In [ ]:
print(x_pos)

In [ ]:
myexplorer2.events_subset.shape

In [ ]:
import pyqtgraph as pg
from pyqtgraph import functions as fn
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl
import numpy as np
'''
# start qt event loop
_instance2 = QApplication.instance()
if not _instance2:
    _instance2 = QApplication([])
app = _instance2
'''
app = pg.mkQApp("GLScatterPlotItem Example")



w = gl.GLViewWidget()
w.show()
w.setWindowTitle('pyqtgraph example: GLScatterPlotItem')
w.setCameraPosition(distance=20)

g = gl.GLGridItem()
w.addItem(g)

pos = np.random.random(size=(100000,3))
pos *= [10,-10,10]

sp2 = gl.GLScatterPlotItem(pos=pos, color=(1,1,1,1), size=10)
w.addItem(sp2)